In [7]:
textFile = sc.textFile("iliad")

textFileTwo= sc.textFile("odyssey")

print "counts"
print textFile.count()
print textFileTwo.count()






counts
63854
65405


In [9]:
#Filter function for fomd the number of times achilles loggedin
def filter_user_achilles(line):
  if "starting session" in line.lower() and "user achille" in line.lower():
    return line


counts = textFile.flatMap(lambda x:[x]) \
		 .filter(lambda x:filter_user_achilles(x)) \
                 .map(lambda x: (1, 1)) \
                 .reduceByKey(lambda x, y: x + y).collect()
            

counts2 = textFileTwo.flatMap(lambda x:[x]) \
		 .filter(lambda x:filter_user_achilles(x)) \
                 .map(lambda x: (1, 1)) \
                 .reduceByKey(lambda x, y: x + y).collect()

print "Achilles"
print counts,counts2


Achilles
[(1, 5173)] [(1, 5228)]


In [13]:
users = textFile.flatMap(lambda x:[x]) \
                     .filter(lambda line:"systemd: starting session " in line.lower() ) \
                     .map(lambda line: str(line.split("user")[-1].strip()[:-1])) \
     		         .distinct()  \
		             .collect()
                    
users2 = textFileTwo.flatMap(lambda x:[x]) \
                .filter(lambda line:"systemd: starting session " in line.lower() ) \
                .map(lambda line: str(line.split("user")[-1].strip()[:-1])) \
     		    .distinct()  \
		        .collect()
print "Users"
print "iliad:",users,"\n","odyssey:",users2              

Users
iliad: ['gaia', 'pollux', 'achille', 'helene', 'hector'] 
odyssey: ['achille', 'hector', 'ares']


In [14]:
session = textFile.flatMap(lambda x:[x]) \
                        .filter(lambda line:"systemd: starting session " in line.lower() ) \
                        .map(lambda line: (str(line.split("user")[-1].strip()[:-1]),1)) \
		                .reduceByKey(lambda x, y: x + y) \
                        .collect()

session2 = textFileTwo.flatMap(lambda x:[x]) \
                        .filter(lambda line:"systemd: starting session " in line.lower() ) \
                        .map(lambda line: (str(line.split("user")[-1].strip()[:-1]),1)) \
		                .reduceByKey(lambda x, y: x + y) \
                        .collect()

print session,"\n",session2

[('gaia', 2), ('pollux', 38), ('achille', 5173), ('helene', 248), ('hector', 9)] 
[('achille', 5228), ('hector', 2), ('ares', 40)]


In [15]:
errors = textFile.flatMap(lambda x:[x]) \
                       .filter(lambda line:"error" in (line.lower()) ) \
                       .map(lambda line: (1,1)) \
                       .reduceByKey(lambda x, y: x + y) \
                       .collect()

errors2 = textFileTwo.flatMap(lambda x:[x]) \
                       .filter(lambda line:"error" in (line.lower()) ) \
                       .map(lambda line: (1,1)) \
                       .reduceByKey(lambda x, y: x + y) \
                       .collect()

print errors,"\n",errors2

[(1, 2723)] 
[(1, 25805)]


In [19]:
error_counts = textFile.flatMap(lambda x:[x]) \
               	     .filter(lambda line:"error" in (line.lower())) \
                     .map(lambda line: (line.split("iliad")[1] if len(line.split("iliad"))>=2 else None,1)) \
		             .filter(lambda line:line is not None) \
                     .reduceByKey(lambda x, y: x + y) \
	                 .sortBy(lambda x: x[1],ascending=False) \
                     .collect()

error_counts2 = textFileTwo.flatMap(lambda x:[x]) \
               	     .filter(lambda line:"error" in (line.lower())) \
                     .map(lambda line: (line.split("odyssey")[1] if len(line.split("odyssey"))>=2 else None,1)) \
		             .filter(lambda line:line is not None) \
                     .reduceByKey(lambda x, y: x + y) \
	                 .sortBy(lambda x: x[1],ascending=False) \
                     .collect()

print error_counts[0:5],"\n",error_counts2[0:5]

[(u' journal: ethtool ioctl error: No such device', 889), (u' gnome-session: ** (evince:31187): WARNING **: Error setting file metadata: No such file or directory', 24), (u' gnome-session: https://yum.dockerproject.org/repo/main/centos/7/repodata/3849c07e5505140b8134d3cf1bef35cd1cfe4797bee74799953d73a568694fd1-filelists.sqlite.bz2: [Errno 14] HTTPS Error 403 - Forbidden', 9), (u" gnome-session: GDBus.Error:org.gtk.GDBus.UnmappedGError.Quark._imsettings_2derror_2dquark.Code5: Current desktop isn't targeted by IMSettings.", 9), (u' firefox.desktop: Crash Annotation GraphicsCriticalError: |[0][GFX1-]: GLContext is disabled due to a previous crash.|[6][GFX1-]: GLContext is disabled due to a previous crash.|[2][GFX1-]: GLContext is disabled due to a previous crash.|[3][GFX1-]: GLContext is disabled due to a previous crash.|[4][GFX1-]: GLContext is disabled due to a previous crash.|[5][GFX1-]: GLContext is disabled due to a previous crash.[GFX1-]: GLContext is disabled due to a previous cras

In [22]:
unique_users = textFile.flatMap(lambda x:[x]) \
               		    .filter(lambda line:"systemd: starting session " in line.lower()) \
               		    .map(lambda line:(str(line.split("user")[-1].strip()[:-1]),"iliad")) \
			            .filter(lambda line:line is not None) \
               		    .distinct() 


unique_users_iliad = textFileTwo.flatMap(lambda x:[x]) \
                                .filter(lambda line:"systemd: Starting Session " in line) \
                                .map(lambda line:(str(line.split("user")[-1].strip()[:-1]),"odyssey")) \
                                .filter(lambda line:line is not None) \
                                .distinct() 


combined_rdd = unique_users.union(unique_users_iliad)

In [23]:
multiple_hosts = combined_rdd.reduceByKey(lambda x,y:x+","+y) \
			   .map(lambda x:x[0] if len(x[1].split(",")) > 1 else None)  \
			   .filter(lambda line:line is not None) \
			   .collect()

print multiple_hosts

['achille', 'hector']


In [24]:
single_login = combined_rdd.reduceByKey(lambda x,y:x+","+y) \
                             .map(lambda x:x if len(x[1].split(","))==1 else None)  \
			                 .filter(lambda line:line is not None) \
                             .collect()

print single_login

[('gaia', 'iliad'), ('pollux', 'iliad'), ('helene', 'iliad'), ('ares', 'odyssey')]


In [25]:
users_sorted_list1 = textFile.flatMap(lambda x:[x]) \
                          .filter(lambda line:"systemd: Starting Session " in line ) \
                          .map(lambda line: str(line.split("user")[-1].strip()[:-1])) \
                          .distinct()  \
		                  .sortBy(lambda x:x) \
                          .collect()

users_sorted_list2 = textFileTwo.flatMap(lambda x:[x]) \
                          .filter(lambda line:"systemd: Starting Session " in line ) \
                          .map(lambda line: str(line.split("user")[-1].strip()[:-1])) \
                          .distinct()  \
		                  .sortBy(lambda x:x) \
                          .collect()


print users_sorted_list1,"\n",users_sorted_list2

['achille', 'gaia', 'hector', 'helene', 'pollux'] 
['achille', 'ares', 'hector']


In [ ]:
#Function to anonymize the user names in log files and write as a separate file
def anonymize_user_names(line,users_sorted_list):
  for user in users_sorted_list:
    if user in line:
      line = line.replace(user,"user-"+str(users_sorted_list.index(user)))
      return line
  return line

anonymised_file1 = textFile.flatMap(lambda x:[x]) \
		                      .map(lambda line: anonymize_user_names(line,users_sorted_list1)) \
			                  .map(lambda line:anonymize_user_names(line,users_sorted_list1) if "su:" in line else line) \
  				              .saveAsTextFile("anonymised-files-"+"iliad")

anonymised_file2 = textFileTwo.flatMap(lambda x:[x]) \
		                      .map(lambda line: anonymize_user_names(line,users_sorted_list2)) \
			                  .map(lambda line:anonymize_user_names(line,users_sorted_list2) if "su:" in line else line) \
  				              .saveAsTextFile("anonymised-files-"+"odyssey")